## Crawling and Extracting Data from Websites

This module uses a set of non-standard libraries, which need to be installed on your machine. By default, your instance should have these installed, but if this is not the case, type these in the Unix shell prompt





In [ ]:
!sudo -H apt-get -y install libxml2-dev libxslt-dev python3-dev

and then

In [ ]:
!sudo -H pip3 install -U lxml

In [ ]:
!sudo -H pip3 install -U pandas

### Fetching the headlines from ESPN.com

Let's start by trying to fetch the headlines from the site ESPN.com.



In [1]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

# Let's start by fetching the page, and parsing it
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

In [2]:
doc

<Element html at 0x7f5a32dc42c8>

Let's start by getting the content of the `<title>` node from the site:

In [4]:
# This will return back a list of nodes that match
# are called <title>......</title>
results = doc.xpath('//title')
results[0].text

'ESPN: The Worldwide Leader in Sports'

The `doc` variable is an `HtmlElement` object, and we can now use **XPath** queries to locate the elements that we need. (Depending on time, we may do in class a tutorial on XPath. For now, you can look at the [W3Schools tutorial](http://www.w3schools.com/xpath/xpath_nodes.asp))

For example, to find all the `<a ...> ... </a>` tags in the returned html, which store the links in the page, we issue the command:

In [5]:
links = doc.xpath("//a")
len(links)

233

In [12]:
links = doc.xpath("//a")
# Iterates over all the links (this means all the nodes
# that matched the //a XPath query) and prints the content
# of the attribute href and the text for that node
for link in links:
    print("==================================")
    print(link.get("href"), "==>", link.text)

# ==> None
/ ==> ESPN
# ==> None
# ==> Scores
/nfl/ ==> NFL
/nba/ ==> NBA
/mlb/ ==> MLB
/college-football/ ==> NCAAF
http://www.espnfc.com ==> Soccer
/nhl/ ==> NHL
# ==> …
/tennis/ ==> Tennis
/golf/ ==> Golf
/mens-college-basketball/ ==> NCAAM
/mma/ ==> MMA
/wwe/ ==> WWE
/boxing/ ==> Boxing
/esports/ ==> esports
/chalk/ ==> Chalk
/analytics/ ==> Analytics
/womens-basketball/ ==> NCAAW
/womens-basketball/ ==> WNBA
/racing/nascar/ ==> NASCAR
/jayski/ ==> Jayski 
/racing/ ==> Racing
/horse-racing/ ==> Horse
http://www.espn.com/college-sports/football/recruiting/ ==> RN FB
http://www.espn.com/college-sports/basketball/recruiting/index ==> RN BB
/college-sports/ ==> NCAA
http://www.espn.com/sports/llws17/story/_/page/LittleLeagueWorldSeries/little-league-world-series-espn ==> LLWS
/olympics/ ==> Olympic Sports
http://www.espn.com/specialolympics/ ==> Special Olympics
http://xgames.com/ ==> X Games
http://espncricinfo.com/ ==> Cricket
http://www.espnscrum.com/ ==> Rugby
/endurance/ ==> Endur

In [8]:
lnk = links[70]
type(lnk)

lxml.html.HtmlElement

The `lnk` variable is again an HtmlElement. To get parts of the html element that we need, we can use the `get` method (e.g., to get the `href` attribute) and the `text_content()` method (to get the text within the `<a>...</a>` tag.

In [9]:
lnk.get("href")

'http://www.espn.com/espn/apps/download?app=espn'

In [ ]:
lnk.text_content()

Now, let's revisit the _list comprehension_ approach that we discussed in the Python Primer session, for quickly constructing lists:

In [ ]:
urls = [lnk.get("href") for lnk in doc.xpath('//a')]
urls

#### Exercise

Use a list compresension approach, to get the text_content of all the URLs in the page.

In [ ]:
# your code here


And now create a list where we put together text content and the URL for each link

In [ ]:
# your code here


#### Let's get the headlines...

Now, let's examine how we can get the data from the website. The key is to understand the structure of the HTML, where the data that we need is stored, and how to fetch the elements. Then, using the appropriate XPath queries, we will get what we want.

In [13]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

Let's start by fetching the page, and parsing it

In [14]:
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

In [15]:
# Find all the <a> nodes that have an attribute
# data-mptype that is equal to 'headline'
results = doc.xpath('//a[@data-mptype="headline"]')
len(results)

26

In [22]:
for result in results:
    print(result.text, '==>', 'http://www.espn.com'+result.get('href'))
    print('=============================')

How to bet Washington at Philadelphia ==> http://www.espn.com/chalk/insider/story/_/id/21111123/nfl-vegas-handicappers-break-how-bet-washington-philadelphia
Vikings' Sendejo suspended for hit on Wallace ==> http://www.espn.com/nfl/story/_/id/21130180/andrew-sendejo-minnesota-vikings-suspended-one-game-hit-mike-wallace-baltimore-ravens
Sources: Lynch appeal heard; Peters testifies ==> http://www.espn.com/nfl/story/_/id/21129992/marshawn-lynch-oakland-raiders-appeal-heard-marcus-peters-testifies
Curry fined $50K over throwing mouthpiece ==> http://www.espn.com/nba/story/_/id/21129455/golden-state-warriors-guard-stephen-curry-fined-50000-throwing-mouthpiece
GM: Bledsoe likely played final game for Suns ==> http://www.espn.com/nba/story/_/id/21127746/eric-bledsoe-sent-home-meeting-phoenix-suns
Amid trade chatter, WR Bryant calls in sick ==> http://www.espn.com/nfl/story/_/id/21128339/martavis-bryant-pittsburgh-steelers-misses-meetings-due-illness
Suns' Jackson: Hand gesture at fan wasn't g

By using the `"Right-Click > Inspect"` option of Chrome,
we right click on the headlines and select `"Inspect"`.
This opens the source code.
There we see that all under a `<div class="headlineStack">` tag.

In [ ]:
headlineNode = doc.xpath('//div[@class="headlineStack"]')

The result of that operation is a list with 9 elements.

In [ ]:
type(headlineNode)

In [ ]:
len(headlineNode)

Each headline is under a  `<li><a href="...."></a>` tag.
So, we get all the `<li><a ...>` tags within the `<div class="headlineStack">`
(which is stored in the "`headlineNode`" variable)

In [ ]:
headlines = headlineNode[0].xpath('.//li/a')
len(headlines)

Now, we have the nodes with the conent in the headlines variable.
We extract the text and the URL.

In [ ]:
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
data

And let's create our dataframe, so that we can have a better view

In [ ]:
dataframe = pandas.DataFrame(data)
dataframe

#### Of course, there are always more than one way to skin a cat...

Alternatively, if we did not want to restrict ourselves to just the first headline box, we could write an alternative query, to get back all the headlines, that appear in an XPath `//div[@class="headlineStack"]//li/a`:

In [ ]:
headlines = doc.xpath('//div[@class="headlineStack"]//li/a')
data = [{"Title": a.text_content(), "URL": "http://www.espn.com" + a.get("href")} for a in headlines]
df = pandas.DataFrame(data)
df

### In Class Example: Crawl BuzzFeed

* We will try to get the top articles that appear on Buzzfeed
* We will grab the link for the article, the text of the title, the description, and the editor.
* The results will be stored in a dataframe (we will see in detail what a dataframe is, in a couple of modules)


In [ ]:
#your code here
import requests
from lxml import html

resp = requests.get("http://www.buzzfeed.com")
doc = html.fromstring(resp.text)

#### Solution for Buzzfeed (as of February 27, 2017)

In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas

# Let's start by fetching the page, and parsing it
url = "http://www.buzzfeed.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

articleNodes = doc.xpath("//div[@class='xs-px05 sm-pl05']") 

def parseArticleNode(article):
    headline = article.xpath(".//a/h2")[0].text_content()
    headline_link = article.xpath(".//a")[0].get("href")
    description = article.xpath(".//a/p")[0].text_content()
    editor = article.xpath(".//a[contains(@class,'card__byline-link')]/span")[0].text_content().strip()
    
    result = {
        "headline": headline,
        "URL" : headline_link,
        "description" : description,
        "editor" : editor
    }
    return result

data = [parseArticleNode(article) for article in articleNodes]
df = pandas.DataFrame(data)
df